In [119]:
import os
import cv2

In [120]:
DATASET_PATH = "dataset"

In [121]:
folders = []
recognizer = cv2.face.LBPHFaceRecognizer_create(2, 2, 7, 7)
classifier = cv2.CascadeClassifier()
classifier.load("utils/lbpcascade_frontalface.xml")

True

In [122]:
import numpy as np
# read all images
images = []
labels = []
for i, folder in enumerate(os.listdir(DATASET_PATH)):
    SUB_PATH = "{}/{}".format(DATASET_PATH, folder)
    if os.path.isdir(SUB_PATH):
        folders.append(folder)
        for image in os.listdir(SUB_PATH):
            img = cv2.imread("{}/{}".format(SUB_PATH, image))
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            face = classifier.detectMultiScale(gray, 1.2, 5)
            if len(face) > 0:
                print("Face Detected: ", image)
                (x, y, w, h) = face[0]
                cropped_image = gray[y:y+h, x:x+w]
                images.append(cropped_image)
                labels.append(i)
                #cv2.imwrite("face.jpg",cropped_image)



Face Detected:  1LM.jpg
Face Detected:  WhatsApp Image 2022-07-26 at 11.58.10.jpeg
Face Detected:  WhatsApp Image 2022-07-26 at 11.58.09 (2).jpeg
Face Detected:  WhatsApp Image 2022-07-26 at 11.58.05.jpeg
Face Detected:  WhatsApp Image 2022-07-26 at 11.58.05 (3).jpeg
Face Detected:  WhatsApp Image 2022-07-26 at 11.58.02 (1).jpeg
Face Detected:  WhatsApp Image 2022-07-26 at 11.58.08 (2).jpeg
Face Detected:  WhatsApp Image 2022-07-26 at 11.58.09 (1).jpeg
Face Detected:  WhatsApp Image 2022-07-26 at 11.58.04 (1).jpeg
Face Detected:  WhatsApp Image 2022-07-26 at 11.58.07 (1).jpeg
Face Detected:  WhatsApp Image 2022-07-26 at 11.58.08 (1).jpeg
Face Detected:  WhatsApp Image 2022-07-26 at 11.58.05 (2).jpeg
Face Detected:  WhatsApp Image 2022-07-26 at 11.58.09.jpeg
Face Detected:  WhatsApp Image 2022-07-26 at 11.58.05 (1).jpeg
Face Detected:  WhatsApp Image 2022-07-26 at 11.58.07 (2).jpeg
Face Detected:  WhatsApp Image 2022-07-26 at 11.58.06 (1).jpeg
Face Detected:  WhatsApp Image 2022-07-26 a

In [123]:
# write persons names to labels file
with open("utils/labels.txt", 'w') as f:
    for i in folders:
        f.write(i)
        f.write("\n")

In [124]:
recognizer.train(np.array(images), np.array(labels))
recognizer.save("utils/embeddings.xml")

/tmp/ipykernel_28137/3216828076.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  recognizer.train(np.array(images), np.array(labels))


In [125]:
labels


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]